In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


____

# 1. Data

In [0]:
df_disaster = pd.read_csv('train.csv')
df_disaster.shape

(7613, 5)

In [0]:
df_disaster.iloc[158].text

'DTN Brazil: Experts in France begin examining airplane debris found on Reunion Island: French air accident exp... http://t.co/M9IG3WQ8Lq'

In [0]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
import spacy
#from spacy import displacy
nlp = spacy.load("en_core_web_sm")

# 2. POS Tagging

Also included are a description of eight parts of speech: noun, verb, pronoun, preposition, adverb, conjunction, participle, and article.  

Parts of speech (also known as POS, word classes, or syntactic categories) are useful because they reveal a lot about a word and its neighbors. Knowing whether a word is a noun or a verb tells us about likely neighboring words (nouns are preceded by determiners and adjectives, verbs by nouns) and syntactic structure (nouns are generally part of noun phrases) 

_Fuente: [https://web.stanford.edu/~jurafsky/slp3/8.pdf](https://web.stanford.edu/~jurafsky/slp3/8.pdf)_

[Spacy annotation scheme for POS](https://spacy.io/api/annotation#pos-tagging)

In [0]:
doc = nlp(df_disaster.iloc[158].text)

for token in doc: 
    print(token.text, token.pos_, token.tag_)

DTN PROPN NNP
Brazil PROPN NNP
: PUNCT :
Experts NOUN NNS
in ADP IN
France PROPN NNP
begin VERB VBP
examining VERB VBG
airplane NOUN NN
debris NOUN NN
found VERB VBN
on ADP IN
Reunion PROPN NNP
Island PROPN NNP
: PUNCT :
French ADJ JJ
air NOUN NN
accident NOUN NN
exp NOUN NN
... PUNCT :
http://t.co/M9IG3WQ8Lq INTJ UH


In [0]:
#displacy.serve(doc, style="dep")

# 3. NER

A **named entity** is, roughly speaking, anything that can be referred to with a proper name: a person, a location, an organization.  

The term is commonly extended to include things that aren’t entities per se, including dates, times, and other kinds of temporal expressions, and even numerical expressions like prices.

_Fuente: [https://web.stanford.edu/~jurafsky/slp3/18.pdf](https://web.stanford.edu/~jurafsky/slp3/18.pdf)_

[Spacy annotation scheme for NER](https://spacy.io/api/annotation#named-entities)

In [0]:
df_disaster.iloc[158].text

'DTN Brazil: Experts in France begin examining airplane debris found on Reunion Island: French air accident exp... http://t.co/M9IG3WQ8Lq'

In [0]:
doc = nlp(df_disaster.iloc[158].text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Brazil 4 10 GPE
France 23 29 GPE
Reunion Island 71 85 LOC
French 87 93 NORP


In [0]:
#displacy.serve(doc, style="ent")

# 4. Text Categorization

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df_disaster['text'], df_disaster['target'], test_size=0.2)

### 4.1 Obtener vectores por tf-idf

In [0]:
#Fuente: https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import re

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lemma_.lower().strip() for word in mytokens if (word not in stop_words) and re.search('[a-z]', word.lemma_.lower())]

    # return preprocessed list of tokens
    return mytokens

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word', tokenizer = spacy_tokenizer)
vectorizer.fit(df_disaster.text)

#obtener vectores
X_train_Tfidf = vectorizer.transform(X_train)
X_val_Tfidf = vectorizer.transform(X_val)

#convertirlos a DataFrame
feature_names = vectorizer.get_feature_names()
X_train_Tfidf = pd.DataFrame(X_train_Tfidf.todense().tolist(), columns=feature_names)
X_val_Tfidf = pd.DataFrame(X_val_Tfidf.todense().tolist(), columns=feature_names)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [0]:
X_train_Tfidf.describe()

,#fire,#pandemonium,#yr,#yycstorm,&amp,':http://t.co/724gq5ebqz,'s,'the,-&gt,--&gt,----&gt,------&gt,--head,--just,--thus,-@entershikari,-a,-anonymous,-d,-david,-honey,-how,-hubert,-in,-iowa,-kfvs12,-legion,-loses,-lou,-me,-of,-on,-over,-pandemonium,-popular,-population:6,-pron-,-runs,-sets,-sigmund,...,û÷bomb,û÷british,û÷devastatedûª,û÷em,û÷exceptionalûª,û÷extremely,û÷faceless,û÷facelessûª,û÷firstûª,û÷food,û÷good,û÷hazardûª,û÷heat,û÷hijacker,û÷hoax,û÷ill,û÷institute,û÷islamic,û÷it,û÷itûªs,û÷leaves,û÷letûªs,û÷minimum,û÷muslim,û÷notherû.,û÷nuclear,û÷plot,û÷politics,û÷ransomwareûª,û÷secondûª,û÷the,û÷vulnerableûª,û÷we,å£100bn,å£279.00end,å£6bn,å¬'only,åèmgn,åêfedex,åêi
count,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.0,6090.000000,6090.000000,6090.0,6090.000000,6090.000000,6090.000000,6090.0,6090.0,6090.0,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.0,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.0,6090.000000,6090.000000,6090.000000,6090.000000,6090.0,6090.000000,6090.0,6090.000000,6090.000000,6090.000000,6090.000000,...,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.0,6090.000000,6090.000000,6090.000000,6090.000000,6090.0,6090.000000,6090.0,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000,6090.0,6090.000000,6090.000000,6090.000000,6090.000000,6090.000000
mean,0.000055,0.000077,0.000078,0.000056,0.000054,0.0,0.000213,0.000054,0.0,0.000095,0.000062,0.000058,0.0,0.0,0.0,0.000067,0.000048,0.000059,0.000055,0.000051,0.000055,0.000059,0.0,0.000052,0.000052,0.000049,0.000055,0.000076,0.0,0.000075,0.000041,0.000061,0.000060,0.0,0.000046,0.0,0.017083,0.000061,0.000087,0.000054,...,0.000060,0.000046,0.000054,0.000063,0.000054,0.000240,0.000056,0.000055,0.000042,0.000063,0.000106,0.000056,0.000053,0.000072,0.000065,0.0,0.000068,0.000053,0.000056,0.000084,0.0,0.000075,0.0,0.000060,0.000039,0.000073,0.000109,0.000261,0.000048,0.000042,0.000048,0.000062,0.000064,0.000050,0.0,0.000059,0.000057,0.000044,0.000054,0.000048
std,0.004272,0.006036,0.006090,0.004362,0.004211,0.0,0.008459,0.004190,0.0,0.005266,0.004804,0.004545,0.0,0.0,0.0,0.005205,0.003778,0.004578,0.004320,0.004005,0.004270,0.004578,0.0,0.004080,0.004080,0.003839,0.004258,0.005914,0.0,0.005829,0.003218,0.004763,0.004718,0.0,0.003593,0.0,0.054238,0.004789,0.006783,0.004223,...,0.004646,0.003604,0.004182,0.004915,0.004238,0.009373,0.004388,0.004326,0.003263,0.004904,0.005862,0.004355,0.004158,0.005595,0.005090,0.0,0.005327,0.004106,0.004355,0.006550,0.0,0.005866,0.0,0.004655,0.003045,0.005698,0.005992,0.009115,0.003763,0.003263,0.003727,0.004805,0.004965,0.003924,0.0,0.004595,0.004441,0.003432,0.004249,0.003755
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

### 4.2 Modelamiento

In [0]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def test_model(train, val, y_train, y_val):
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(train,y_train)

    predictions = Naive.predict(val)
    print("Accuracy Score -> ",accuracy_score(predictions, y_val)*100)
    print("F1 Score -> ",f1_score(predictions, y_val)*100)

In [0]:
test_model(X_train_Tfidf, X_val_Tfidf, y_train, y_val)

Accuracy Score ->  80.69599474720945
F1 Score ->  73.7030411449016


### 4.3 Agregar información sobre NER

In [0]:
def spacy_ents(text):
    doc = nlp(text)
    ents = {'PERSON':0,'ORG':0,'GPE':0,'LOC':0}
    for ent in doc.ents:
        if(ent.label_ in ents.keys()): ents[ent.label_] = 1
    return ents

In [0]:
#Obtener las entidades
X_train_ents = pd.DataFrame(list(X_train.apply(spacy_ents)))
X_val_ents = pd.DataFrame(list(X_val.apply(spacy_ents)))

In [0]:
X_train_ents.sample(10)

,PERSON,ORG,GPE,LOC
4926,0,0,0,0
5101,0,0,0,0
1576,0,1,0,0
4439,0,0,0,0
712,0,0,0,0
732,1,0,0,0
3070,0,0,0,0
3966,0,0,0,0
1652,1,1,0,0
6004,0,0,0,0


In [0]:
X_train_ents.describe()

,PERSON,ORG,GPE,LOC
count,6090.000000,6090.000000,6090.000000,6090.000000
mean,0.212151,0.368309,0.196388,0.032512
std,0.483239,0.668470,0.464386,0.181037
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,0.000000,0.000000
max,4.000000,7.000000,4.000000,2.000000


In [0]:
#Concatenarlo con los vectores de tf-idf
X_train_tfents = pd.concat([X_train_Tfidf, X_train_ents], axis=1)
X_val_tfents = pd.concat([X_val_Tfidf, X_val_ents], axis=1)

In [0]:
#Probar un modelo
test_model(X_train_tfents, X_val_tfents, y_train, y_val)

Accuracy Score ->  79.44845699277742
F1 Score ->  72.99396031061261
